In [65]:
import csv
import statistics as stat
from scipy.signal import find_peaks
import os


In [66]:
#####Think about the characteristics for machine learning
#####add functions below

In [67]:
def Find_uuid_data_and_time(f):#####  find uuid of thingy
    data = []
    time = []
    for row in f.readlines():
        if 'uuid' in row:
            uuid = row.split()[1].split('"')[3]
        if 'TVOC' in row:
            data.append(int(row.split(',')[1]))
            time.append(row.split(',')[2].replace('\n', ''))
    return uuid,data,time

In [68]:
def devide(data,time,start_time):
    flag = 0
    count = -1
    count_dev = 0
    data_dev = [[],[],[],[]]
    time_dev = [[],[],[],[]]
    start_time = start_time.split(':')
    start_time = int(start_time[-1])+int(start_time[-2])*60+int(start_time[-3])*3600
    for t in time:
        count += 1
        if flag == 0:
            flag = 1
            data_dev[count_dev].append(data[count])
            time_dev[count_dev].append(int(time[count].split(':')[-1])+int(time[count].split(':')[-2])*60+int(time[count].split(':')[-3])*3600-start_time)
        else:
            B=t.split(':')
            B = int(B[-1])+int(B[-2])*60+int(B[-3])*3600
            if B - start_time > 300*(count_dev+1) or count == len(time)-1:
                flag = 0
                count_dev += 1
                if count_dev == 4:
                    break
                continue
            else :
                data_dev[count_dev].append(data[count])
                time_dev[count_dev].append(int(time[count].split(':')[-1])+int(time[count].split(':')[-2])*60+int(time[count].split(':')[-3])*3600-start_time)         
   
    return data_dev,time_dev

In [69]:
def Find_max_min_avg_median(data):
    dmax = max(map(int,data))
    dmin = min(map(int,data))
    davg = (dmax+dmin)/2
    dmedian = stat.median(map(int,data))
    return dmax,dmin,davg,dmedian

In [70]:
def Find_time(value,data,time):
    timevalue = []
    timelist = []
    for i in range(len(value)):
        for j in range(len(data)):
            if data[j] == value[i]:
                timevalue.append(time[j])
        timelist.append(timevalue)
    return timelist

In [71]:
def Find_var(data):  ## find variance
    var = stat.pvariance(map(int,data))
    return var

In [72]:
def Find_time_interval(timeA,timeB):
    A=timeA.split(':')
    B=timeB.split(':')
    A = int(A[-1])+int(A[-2])*60+int(A[-3])*3600
    B = int(B[-1])+int(B[-2])*60+int(B[-3])*3600  
    if(A>B):
        interval = A-B
    else:
        interval = B-A
    return interval

In [73]:
## find no of peaks, their value and time(even though multiple peaks should not happen)
def Find_peaks_and_prominence(data,time,davg):## use avg to decide is not good, need better way
    peak_id,peaks_property = find_peaks(data,height = davg,threshold = None, distance = 50, prominence = davg/2, width =10, wlen = None,rel_height =0.5, plateau_size = None)
    value = 0
    timelist = 0
    prominence = 0
    for i in range(len(peak_id)):
        if data[peak_id[i]] > value:
            value = data[peak_id[i]]
            timelist = time[peak_id[i]]
            prominence = peaks_property["prominences"][i]
    
    return value,timelist,prominence

In [74]:

def Find_char(filename,start_time):
    with open(filename, 'r') as f:
        uuid,data,time = Find_uuid_data_and_time(f)
        #data,time = data_and_time(f)
    #data,time = data_manipulation(data,time)
    #print(data)
    data_dev, time_dev = devide(data,time,start_time)
   # print(uuid,data_dev, time_dev)
    dmax = [0] * 4
    dmin = [0] * 4
    davg = [0] * 4
    dmedian = [0] * 4
    peak_values= [0] * 4
    peak_times= [0] * 4
    prominence= [0] * 4
    dvar = [0] * 4
    for i in range(0,4):
        
        dmax[i],dmin[i],davg[i],dmedian[i] = Find_max_min_avg_median(data_dev[i])
        peak_values[i],peak_times[i],prominence[i] = Find_peaks_and_prominence(data_dev[i],time_dev[i],davg[i])
        dvar[i] = Find_var(data_dev[i])
    #interval = Find_time_interval(time[0],time[-1])
    #get region
    region = filename.split('/')[2]
    location_x = filename.split('/')[3].split('_')[2]
    location_y = filename.split('/')[3].split('_')[3]
    f.close
    export(uuid,dmax,dmin,davg,dmedian,peak_values,peak_times,prominence,dvar,region,location_x,location_y)
    #return uuid,dmax,dmin,davg,dmedian,peaks_num,peak_values,peak_times,prominence,dvar,region,location


In [75]:

def export(uuid,dmax,dmin,davg,dmedian,peak_values,peak_times,prominence,dvar,region,location_x,location_y):
    with open('all_1.csv', mode='a',newline='') as overall_file:
        fieldnames = ['sensor',
                      'dmax','dmin','davg','dmedian','peak_values','peak_times','prominence','dvar',
                      'Region','Location_x','Location_y','period']
        writer = csv.DictWriter(overall_file, fieldnames=fieldnames)
        for i in range(0,4):
            writer.writerow({'sensor':uuid,
                         'dmax':dmax[i],'dmin':dmin[i],'davg':davg[i],'dmedian':dmedian[i],
                         'peak_values':peak_values[i],'peak_times':peak_times[i],'prominence':prominence[i],'dvar':dvar[i],
                         'Region':region,'Location_x':location_x,'Location_y':location_y,'period': i })
    overall_file.close
    return   

In [76]:
#read all files
def read_all_files(directory):
    entries = os.listdir(directory)


    for foldernames in entries:
        subfolder = os.listdir(directory+'/'+ str(foldernames))
        print(foldernames)
        for subfoldernames in subfolder:
            files= os.listdir(directory+'/'+ str(foldernames)+'/'+ str(subfoldernames))
            flag = 0
            for filenames in files:
                if flag == 0:
                    with open (directory+'/'+ str(foldernames)+'/'+ str(subfoldernames)+'/'+str(filenames),'r') as f:
                        uuid, data, time = Find_uuid_data_and_time(f)
                    start_time = time [0]
                    flag = 1
                    f.close
                    
                Find_char(directory+'/'+ str(foldernames)+'/'+ str(subfoldernames)+'/'+str(filenames),start_time)

    return
    

In [84]:
def modify(filename):
    new_row_0 =[0]*31
    new_row_1 =[0]*31
    new_row_2 =[0]*31
    new_row_3 =[0]*31
    with open(filename,'r')as f1:
        with open('all_2.csv', mode='a',newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            flag_0 = 0
            flag_1 = 0
            flag_2 = 0
            for row in f1.readlines():
                uuid = row.split(',')[0]
                if uuid == 'e2c84430970e':
                    if flag_0 == 0:
                        new_row_0[0:9] = row.split(',')[0:9]
                    if flag_0 == 1:
                        new_row_1[0:9] = row.split(',')[0:9]
                    if flag_0 == 2:
                        new_row_2[0:9] = row.split(',')[0:9]
                    if flag_0 == 3:
                        new_row_3[0:9] = row.split(',')[0:9]
                    flag_0 += 1
                elif uuid == 'e062039dd423':
                    if flag_1 == 0:
                        new_row_0[9:18] = row.split(',')[0:9]
                    if flag_1 == 1:
                        new_row_1[9:18] = row.split(',')[0:9]
                    if flag_1 == 2:
                        new_row_2[9:18] = row.split(',')[0:9]
                    if flag_1 == 3:
                        new_row_3[9:18] = row.split(',')[0:9]
                    flag_1 += 1
                else:
                    if flag_2 == 0: 
                        new_row_0[18:30] = row.split(',')[0:12]
                        new_row_0[30] = int(row.split(',')[12])
                    if flag_2 == 1: 
                        new_row_1[18:30] = row.split(',')[0:12]
                        new_row_1[30] = int(row.split(',')[12])
                    if flag_2 == 2: 
                        new_row_2[18:30] = row.split(',')[0:12]
                        new_row_2[30] = int(row.split(',')[12])
                    if flag_2 == 3: 
                        new_row_3[18:30] = row.split(',')[0:12]
                        new_row_3[30] = int(row.split(',')[12])
                    flag_2 += 1
                if flag_0 == 4 and flag_1 == 4 and flag_2 == 4 :
                    flag_0 = 0
                    flag_1 = 0
                    flag_2 = 0
                    writer.writerow(new_row_0)
                    writer.writerow(new_row_1)
                    writer.writerow(new_row_2)
                    writer.writerow(new_row_3)
                    new_row_0 =[0]*31
                    new_row_1 =[0]*31
                    new_row_2 =[0]*31
                    new_row_3 =[0]*31
        csvfile.close
    f1.close
    return

In [82]:
read_all_files("./DATA")

Region 1


PermissionError: [Errno 13] Permission denied: 'all_1.csv'

In [85]:
modify('all_1.csv')